

References:
Kaggle Data (not used):
https://www.kaggle.com/san-francisco/sf-restaurant-scores-lives-standard

SF Website, data downloaded Apr 6, 2019
https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i


https://www.yelp.com/developers/api_terms
Allowable non-commercial use of the Yelp Content. Notwithstanding the foregoing, you may use the Yelp Content to perform certain analysis for non-commercial uses only, such as creating rich visualizations or exploring trends and correlations over time, so long as the underlying Yelp Content is only displayed in the aggregate as an analytical output, and not individually. For example, this is an acceptable non-commercial analytical use of the Yelp Content. “Non-commercial use” means any use of the Yelp Content which does not generate promotional or monetary value for the creator or the user, or such use does not gain economic value from the use of our content for the creator or user, i.e. you. If you are interested in commercial use of the Yelp Content, please visit www.yelp.com/knowledge.


https://github.com/Yelp/yelp-fusion/blob/master/fusion/python/sample.py


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import glob, os 

%matplotlib inline

In [6]:
df = pd.read_csv('Restaurant_Scores_-_LIVES_Standard.csv')
df_restaurants = pd.read_csv('yelp_info.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53854 entries, 0 to 53853
Data columns (total 17 columns):
business_id              53854 non-null int64
business_name            53854 non-null object
business_address         53854 non-null object
business_city            53854 non-null object
business_state           53854 non-null object
business_postal_code     52568 non-null object
business_latitude        29567 non-null float64
business_longitude       29567 non-null float64
business_location        29567 non-null object
business_phone_number    16856 non-null float64
inspection_id            53854 non-null object
inspection_date          53854 non-null object
inspection_score         39742 non-null float64
inspection_type          53854 non-null object
violation_id             40744 non-null object
violation_description    40744 non-null object
risk_category            40744 non-null object
dtypes: float64(4), int64(1), object(12)
memory usage: 7.0+ MB


In [21]:
df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,68454,Kippu Japanese Restaurant,1707 BUCHANAN St,San Francisco,CA,94115,37.785639,-122.429841,"(37.785639, -122.429841)",NaN,68454_20190128,01/28/2019 12:00:00 AM,NaN,Complaint,68454_20190128_103114,High risk vermin infestation,High Risk
1,67448,Soo Fong Restaurant,3801 03rd St 370,San Francisco,CA,94124,37.741829,-122.388098,"(37.741829, -122.388098)",NaN,67448_20180507,05/07/2018 12:00:00 AM,92.0,Routine - Unscheduled,67448_20180507_103149,Wiping cloths not clean or properly stored or ...,Low Risk
2,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,1757_20170928,09/28/2017 12:00:00 AM,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
3,93022,Wise Sons Delicatessen,537 Octavia St,San Francisco,CA,94102,NaN,NaN,NaN,NaN,93022_20180907,09/07/2018 12:00:00 AM,92.0,Routine - Unscheduled,93022_20180907_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk
4,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,4864_20161206,12/06/2016 12:00:00 AM,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk


In [5]:
df.business_id.nunique()

6179

In [19]:
# For Python 3.0 and later
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

API_KEY = # Fill in API key here

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/search'  # Business ID will come after slash.
#REVIEWS_PATH = 'v3/businesses/{id}/reviews'

city = 'San Francisco'
state = 'CA'
country = 'US'

SEARCH_LIMIT = 1


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    #url = '{0}{1}'.format(host, path)
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    print(u'Querying {0} ...'.format(url))
    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, restuarant, address, zipcode):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': restuarant.replace(' ', '+'),
        'location': address.replace(' ', '+') + '+' + city.replace(' ', '+') + '+' + str(zipcode),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, BUSINESS_PATH, api_key, url_params=url_params)

def reviews(api_key, restaurant_id):
    reviews_path = '/v3/businesses/' + restaurant_id + '/reviews'
    #print(reviews_path)
    return request(API_HOST, reviews_path, api_key)

In [9]:
trial = reviews(API_KEY, 'NcRzv0ZLoFKPvhvTIQa8PQ')

/v3/businesses/NcRzv0ZLoFKPvhvTIQa8PQ/reviews
Querying https://api.yelp.com/v3/businesses/NcRzv0ZLoFKPvhvTIQa8PQ/reviews ...


In [13]:
trial2 = pd.DataFrame.from_dict(reviews(API_KEY, df_restaurants.id[0])['reviews'])
trial2

/v3/businesses/NcRzv0ZLoFKPvhvTIQa8PQ/reviews
Querying https://api.yelp.com/v3/businesses/NcRzv0ZLoFKPvhvTIQa8PQ/reviews ...


,id,rating,text,time_created,url,user
0,PIr0AMHVb5KyzW82xEajyg,5,Thanksgiving meal is FANTASTIC!!!! Well worth ...,2018-11-25 22:50:56,https://www.yelp.com/biz/whole-foods-market-sa...,"{'id': 'qIJGy4K8Mh8WgWWwFuOctA', 'profile_url'..."
1,_4utFYBvJE8ci5Yvg4vYFQ,2,"I love Whole Foods as a chain, but this is pro...",2018-12-01 14:06:37,https://www.yelp.com/biz/whole-foods-market-sa...,"{'id': '87eCDML2EQt-S-j92WwwDg', 'profile_url'..."
2,8qMcPQg4Nev_DXdGUdJY1A,1,This is a review for the bakery department onl...,2018-11-08 16:23:37,https://www.yelp.com/biz/whole-foods-market-sa...,"{'id': 'slYCZqo1VcM9nUNVFUtf3g', 'profile_url'..."


In [23]:
trial2.to_csv('rest_reviews_test' + '.csv')

In [23]:
#business_ids = [0, 10]
business_ids = list(df.business_id.unique())
#Use a mini version of dataframe for troubleshooting
#df_mini = df.iloc[0:2]

for row in df.itertuples():
    name = row.business_name
    address = row.business_address
    zipcode = row.business_postal_code
    business_id = row.business_id
    if business_id not in business_ids:
        try:
            business_ids.append(business_id)
            new_df = pd.DataFrame.from_dict(search(API_KEY, name, address, zipcode)['businesses'])
            new_df['business_id'] = business_id
            new_df.to_csv('spare' + 'restaurant' + str(business_id) + '.csv')
            #df_rest.append(new_df)
            print('yay')
        except:
            print(name + ' error')

Querying https://api.yelp.com/v3/businesses/search ...
yay
Querying https://api.yelp.com/v3/businesses/search ...
yay


In [24]:
for row in df_restaurants.itertuples():
    rest_id = row.id
    business_id = row.business_id
    try: 
        new_df = pd.DataFrame.from_dict(reviews(API_KEY, rest_id)['reviews'])
        new_df.to_csv('restaurant_csv/' + 'rest_reviews' + str(business_id) + '.csv')
    except: 
        print(business_id + ' error')

Querying https://api.yelp.com/v3/businesses/NcRzv0ZLoFKPvhvTIQa8PQ/reviews ...
Querying https://api.yelp.com/v3/businesses/jA19LhDjlhgekIHosBtkkQ/reviews ...
Querying https://api.yelp.com/v3/businesses/NfZKuDovZOIbCLZJGt4m2w/reviews ...
Querying https://api.yelp.com/v3/businesses/sT8TRzofJym5w0khZkOJhw/reviews ...
Querying https://api.yelp.com/v3/businesses/jqEpBbf_qNfWuz_GBkwCjw/reviews ...
Querying https://api.yelp.com/v3/businesses/HCTOtXfbGDTcYsnE_gl35Q/reviews ...
Querying https://api.yelp.com/v3/businesses/y0mszyBB6VSyj1rZ-cxGnA/reviews ...
Querying https://api.yelp.com/v3/businesses/t59A7qRhxh8nifGAqc9zlg/reviews ...
Querying https://api.yelp.com/v3/businesses/jwVyTCiKPUkqYPIXwEFgsA/reviews ...
Querying https://api.yelp.com/v3/businesses/agq8VUqVQ_Y3dZh6-IjeIg/reviews ...
Querying https://api.yelp.com/v3/businesses/VD527WwvrhE3Nhzf5vaMNw/reviews ...
Querying https://api.yelp.com/v3/businesses/B0lqvrhX9BNPMggTFmFj8g/reviews ...
Querying https://api.yelp.com/v3/businesses/IhSxq9nv

Querying https://api.yelp.com/v3/businesses/HaAb_L9zVNLjJwrBkSf_Vw/reviews ...
Querying https://api.yelp.com/v3/businesses/0Sh-XNVgGDRwiN4K3UEQJA/reviews ...
Querying https://api.yelp.com/v3/businesses/1dwJK21zm7cuhUKdROWJXQ/reviews ...
Querying https://api.yelp.com/v3/businesses/2Fkv4DJLsjaaRUuaqcsigw/reviews ...
Querying https://api.yelp.com/v3/businesses/nik97G8-H0rFoJKO4X3ZIA/reviews ...
Querying https://api.yelp.com/v3/businesses/VRD9utHL4Ve6jGe8f0MFAA/reviews ...
Querying https://api.yelp.com/v3/businesses/cxwUE8vfZDFFvUwq1J_M3w/reviews ...
Querying https://api.yelp.com/v3/businesses/lJAGnYzku5zSaLnQ_T6_GQ/reviews ...
Querying https://api.yelp.com/v3/businesses/tM6g-7Yh2zuqOalPw2_itw/reviews ...
Querying https://api.yelp.com/v3/businesses/J9UxQxX91mgYF7c2HqRAsA/reviews ...
Querying https://api.yelp.com/v3/businesses/3jXC8RbeYZ4X5c7g5lMdaA/reviews ...
Querying https://api.yelp.com/v3/businesses/cwBxgW9BygQa6_k96H_j4g/reviews ...
Querying https://api.yelp.com/v3/businesses/ZvIpOP6c

Querying https://api.yelp.com/v3/businesses/YZg2OQSxeZbUY0XmsJpUqA/reviews ...
Querying https://api.yelp.com/v3/businesses/8ZmEszI4ZMncK9kuJYQjuA/reviews ...
Querying https://api.yelp.com/v3/businesses/gII3zPsm_Le7OFFTFeJNEQ/reviews ...
Querying https://api.yelp.com/v3/businesses/y0mszyBB6VSyj1rZ-cxGnA/reviews ...
Querying https://api.yelp.com/v3/businesses/2iJ_mVjWB1Ps0qhcuyaqhw/reviews ...
Querying https://api.yelp.com/v3/businesses/mdVq5YJoRhbQl2DD9BPQHw/reviews ...
Querying https://api.yelp.com/v3/businesses/lNbKeOfCMTNkoihZHqrbrg/reviews ...
Querying https://api.yelp.com/v3/businesses/U0AD-wyPJtIkgnfFt-ihAw/reviews ...
Querying https://api.yelp.com/v3/businesses/ziVej4oW2JpUConTE_Y_mg/reviews ...
Querying https://api.yelp.com/v3/businesses/LYExDwZBa1hmKCXXdJ_0Mw/reviews ...
Querying https://api.yelp.com/v3/businesses/O8DgC3zm1fZf34a16YCjDw/reviews ...
Querying https://api.yelp.com/v3/businesses/C36BK5luxi-8apVMMhsizQ/reviews ...
Querying https://api.yelp.com/v3/businesses/muV3xtkB

Querying https://api.yelp.com/v3/businesses/5NvXIkNdCCqUb235WVfMJg/reviews ...
Querying https://api.yelp.com/v3/businesses/u2qFhVAiH3kQIGhEW6PmMA/reviews ...
Querying https://api.yelp.com/v3/businesses/RAqykee1WPsTsEBuBVC3mg/reviews ...
Querying https://api.yelp.com/v3/businesses/AzMWfNSAv7s1rrtLGRldTg/reviews ...
Querying https://api.yelp.com/v3/businesses/MM0HSmoMrJy73fFXPfmz4Q/reviews ...
Querying https://api.yelp.com/v3/businesses/tQE4SPlX9ZlFSbF4JPU4pg/reviews ...
Querying https://api.yelp.com/v3/businesses/7WiKJGM9kPc2WCh0v16Lqw/reviews ...
Querying https://api.yelp.com/v3/businesses/3zohbNdShI5D-2I0YJskUA/reviews ...
Querying https://api.yelp.com/v3/businesses/t0nXh1Nj5WIhg2CZDGyktA/reviews ...
Querying https://api.yelp.com/v3/businesses/E5AaSzcx-LTGT2Iv2oX_Bg/reviews ...
Querying https://api.yelp.com/v3/businesses/udYo3BZ0Y3nR1iOtk58Inw/reviews ...
Querying https://api.yelp.com/v3/businesses/iU-XQ4dhII3WF1UKR4XrNQ/reviews ...
Querying https://api.yelp.com/v3/businesses/lXJA0j9u

Querying https://api.yelp.com/v3/businesses/YV9PYdUD0O4sLDz1DUk0Gw/reviews ...
Querying https://api.yelp.com/v3/businesses/OSZ6bFeCcv4PvUmvuiMEJw/reviews ...
Querying https://api.yelp.com/v3/businesses/pvD64-V2OjBUFlFs4BQ9xA/reviews ...
Querying https://api.yelp.com/v3/businesses/JU3t0Bae1G4p4NVM0sS0Xg/reviews ...
Querying https://api.yelp.com/v3/businesses/jAvnj60BFrMBHLUKtJvOew/reviews ...
Querying https://api.yelp.com/v3/businesses/VVNy1HlymjSif-YzuashuQ/reviews ...
Querying https://api.yelp.com/v3/businesses/8-zW9T7u9L-TDVfJyLMOZw/reviews ...
Querying https://api.yelp.com/v3/businesses/52dNr0tRrwpmumm8Ey5EDg/reviews ...
Querying https://api.yelp.com/v3/businesses/7JGt7QAswGjXNuGN6YYxJQ/reviews ...
Querying https://api.yelp.com/v3/businesses/BqUUe0Bs3RIj8lS_51AhPw/reviews ...
Querying https://api.yelp.com/v3/businesses/2PRgMkqX2dRrcuaJEi_1Rg/reviews ...
Querying https://api.yelp.com/v3/businesses/g_KkVWuIs3QDwnjHRCLN4g/reviews ...
Querying https://api.yelp.com/v3/businesses/E4zEQ-_w

...

Querying https://api.yelp.com/v3/businesses/t4ewGufXDMsY1hQQClNQfg/reviews ...
Querying https://api.yelp.com/v3/businesses/D8-lVk1RQPPAapzVHvJCzg/reviews ...
Querying https://api.yelp.com/v3/businesses/bGI_oq-DVPo2bWJZxDf9gw/reviews ...
Querying https://api.yelp.com/v3/businesses/WSyS5OeKN9C8KgFOMlIujQ/reviews ...
Querying https://api.yelp.com/v3/businesses/UjjNgzBFAVXF3iVXPVjH_Q/reviews ...
Querying https://api.yelp.com/v3/businesses/CZSKlf9kavFr6gYc_UYxVQ/reviews ...
Querying https://api.yelp.com/v3/businesses/saMhj8Co8zxIVGHHmKftqg/reviews ...
Querying https://api.yelp.com/v3/businesses/si_o8bmymED9RWxPHYnZ1A/reviews ...
Querying https://api.yelp.com/v3/businesses/m2VZS2WV75xoZhDRRBWFaw/reviews ...
Querying https://api.yelp.com/v3/businesses/SEuHSFDOoux4MqusTQPNMw/reviews ...
Querying https://api.yelp.com/v3/businesses/xSLGnEGxO85EAk1DXhB4aQ/reviews ...
Querying https://api.yelp.com/v3/businesses/4Nkmr_wTiD1FTjwC6_getA/reviews ...
Querying https://api.yelp.com/v3/businesses/zBn4XAHs

Querying https://api.yelp.com/v3/businesses/SdI0uCelI4ihCwGyMLv03A/reviews ...
Querying https://api.yelp.com/v3/businesses/VZwe5P-at3JYNCxOVSfe-g/reviews ...
Querying https://api.yelp.com/v3/businesses/OvBC83vOK6vRw175vHiArg/reviews ...
Querying https://api.yelp.com/v3/businesses/rfl4M4Twa8O7WqSGAhIIgQ/reviews ...
Querying https://api.yelp.com/v3/businesses/lJAGnYzku5zSaLnQ_T6_GQ/reviews ...
Querying https://api.yelp.com/v3/businesses/AotGpR8U-jdiJEHgrTi4jg/reviews ...
Querying https://api.yelp.com/v3/businesses/kQ_ItLkdUDd7cBQRctlOPQ/reviews ...
Querying https://api.yelp.com/v3/businesses/RRAGptENFdEKokXXc4U4lA/reviews ...
Querying https://api.yelp.com/v3/businesses/vcg3U2vEFuSSK00Njxwk2g/reviews ...
Querying https://api.yelp.com/v3/businesses/UU10C7_zKoZZzAvOvz96oQ/reviews ...
Querying https://api.yelp.com/v3/businesses/HfOv2Z0OyKVw_jt1KnXoag/reviews ...
Querying https://api.yelp.com/v3/businesses/eOCVoXsOEnbr9c89RD0UEQ/reviews ...
Querying https://api.yelp.com/v3/businesses/TmXkYM0N

Querying https://api.yelp.com/v3/businesses/hA1B0lQFci2Yv_7gnify6Q/reviews ...
Querying https://api.yelp.com/v3/businesses/ZuxdNS2GL1u01P-fcOY87g/reviews ...
Querying https://api.yelp.com/v3/businesses/s-ihKanDRDHTD8U49HFA5Q/reviews ...
Querying https://api.yelp.com/v3/businesses/_wwK7GNlzZFoZNyQRygu_w/reviews ...
Querying https://api.yelp.com/v3/businesses/BpFYTc8r8S1FHfBoCyxRSg/reviews ...
Querying https://api.yelp.com/v3/businesses/nBo7CETnSyrzIoGbRnhuJA/reviews ...
Querying https://api.yelp.com/v3/businesses/TwAMgAYuDdqrINjxtO6b8g/reviews ...
Querying https://api.yelp.com/v3/businesses/_cvRHKYUT9lSTEoKhPCUuQ/reviews ...
Querying https://api.yelp.com/v3/businesses/pXV02Lj-jWt6rgHRQzJmqA/reviews ...
Querying https://api.yelp.com/v3/businesses/mk2WW5bA-0WfQP1Uo17_Sg/reviews ...
Querying https://api.yelp.com/v3/businesses/E6wgv3ieF_mR3AM5FhNyCw/reviews ...
Querying https://api.yelp.com/v3/businesses/N8WqaCKEzEDuVFuRN1y3wg/reviews ...
Querying https://api.yelp.com/v3/businesses/mvpDb23K

Querying https://api.yelp.com/v3/businesses/LP-1AKyY6tFkUvmd1p11-Q/reviews ...
Querying https://api.yelp.com/v3/businesses/zFRaJAKrDHiVv4iF5qu4TA/reviews ...
Querying https://api.yelp.com/v3/businesses/56gj15LDt8mcDIeKCbu-tg/reviews ...
Querying https://api.yelp.com/v3/businesses/ywmZI0cSSaBhu3U6DKIMFw/reviews ...
Querying https://api.yelp.com/v3/businesses/7gZvNgv1iV8XRuUqh3UtSA/reviews ...
Querying https://api.yelp.com/v3/businesses/_le519RYA8LPCreBeFunoQ/reviews ...
Querying https://api.yelp.com/v3/businesses/Lr-uGQ8VxaYLEIKTLmA2qQ/reviews ...
Querying https://api.yelp.com/v3/businesses/N7pLMJZyx2i_35vskjk85Q/reviews ...
Querying https://api.yelp.com/v3/businesses/b3AQeuuQw0_o-Qh7i3v28g/reviews ...
Querying https://api.yelp.com/v3/businesses/u-obmxevldBOidDYkL8J6w/reviews ...
Querying https://api.yelp.com/v3/businesses/gLp7o0zddM2bpHHgl3IrTA/reviews ...
Querying https://api.yelp.com/v3/businesses/dPwYKPV-IpYyQiR-VHyuEQ/reviews ...
Querying https://api.yelp.com/v3/businesses/kGG66pu2

Querying https://api.yelp.com/v3/businesses/e3gcKXKhSFYs5fn9FoeG2A/reviews ...
Querying https://api.yelp.com/v3/businesses/ku0E-UT2OdW_vC2JHWgtrA/reviews ...
Querying https://api.yelp.com/v3/businesses/y0mszyBB6VSyj1rZ-cxGnA/reviews ...
Querying https://api.yelp.com/v3/businesses/A4_bL4D2yKd5D8D7QgCF4Q/reviews ...
Querying https://api.yelp.com/v3/businesses/mRvr3toGkIeaUwI5RtW0Gw/reviews ...
Querying https://api.yelp.com/v3/businesses/YYQX-gJhJ6cFzzIhRWjaFA/reviews ...
Querying https://api.yelp.com/v3/businesses/eDP8OzpjR1d1HTsg85Bntw/reviews ...
Querying https://api.yelp.com/v3/businesses/GcvOqnJDNPxmBfTy_SGVTw/reviews ...
Querying https://api.yelp.com/v3/businesses/gqVl3RprESEqkIPeJH0yOg/reviews ...
Querying https://api.yelp.com/v3/businesses/nNxR_L7zdLfVlPiqzNJivg/reviews ...
Querying https://api.yelp.com/v3/businesses/rE4ZPbMuRMiPbgsJmFIVlw/reviews ...
Querying https://api.yelp.com/v3/businesses/j-TcMVPvo1TJwffihAO_8Q/reviews ...
Querying https://api.yelp.com/v3/businesses/5NvXIkNd

In [62]:
df_test.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,kippu-japanese-cuisine-san-francisco,"[{'alias': 'japanese', 'title': 'Japanese'}, {...","{'latitude': 37.7857284545898, 'longitude': -1...",(415) 346-1212,30.553244,sn4ZKOYSdMG3mcT7lPx5xA,https://s3-media1.fl.yelpcdn.com/bphoto/9XaBH6...,False,"{'address1': '1707 Buchanan St', 'address2': '...",Kippu Japanese Cuisine,+14153461212,$$,3.0,364,[pickup],https://www.yelp.com/biz/kippu-japanese-cuisin...


# Create restaurants csv
- Combine this with the poorly named ones in main folder
- Used to create yelp_info.csv

In [15]:
df_check2 = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "restaurant_csv/restaurant*.csv"))), sort=True)

# Combine Restaurant Review Files
- These are the review files
- Saved to restaurant_reviews.csv
- new is the filename, contains business id of restaurant

In [25]:
files = glob.glob(os.path.join('', 'restaurant_csv/rest_reviews*.csv'))
df_reviews = pd.concat([pd.read_csv(fp).assign(new=os.path.basename(fp).split('.')[0]) for fp in files])
df_reviews.to_csv('restaurant_reviews.csv')

In [26]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311 entries, 0 to 2
Data columns (total 8 columns):
Unnamed: 0      8311 non-null int64
id              8311 non-null object
rating          8311 non-null int64
text            8311 non-null object
time_created    8311 non-null object
url             8311 non-null object
user            8311 non-null object
new             8311 non-null object
dtypes: int64(2), object(6)
memory usage: 584.4+ KB


In [27]:
df_reviews.new.nunique()

2795

In [28]:
df_restaurants.business_id.nunique()

2795

In [29]:
df_reviews.head()

,Unnamed: 0,id,rating,text,time_created,url,user,new
0,0,V1kfikCTAefoyr2K1ljw5Q,5,"I love this place.\n\n\n\nAs of this visit, 5 ...",2018-10-31 18:18:30,https://www.yelp.com/biz/the-beer-hall-san-fra...,"{'id': '4kRxFXdgOulK49RaVVEYow', 'profile_url'...",rest_reviews69249
1,1,B0xxe0oaVd4iakwPbhU5xg,3,Great little place with a nice selection of be...,2018-10-22 08:16:28,https://www.yelp.com/biz/the-beer-hall-san-fra...,"{'id': 'jHreH36KMTpW0eNHA0OexA', 'profile_url'...",rest_reviews69249
2,2,Yd-tAtLR1lIeiO1ugsi4lg,4,"Heaven on earth! Endless taps, diverse options...",2019-03-05 17:53:40,https://www.yelp.com/biz/the-beer-hall-san-fra...,"{'id': 'jJo2YqOoKYTz2sUFBfLChw', 'profile_url'...",rest_reviews69249
0,0,iF2hUCKl1KPT359abomQ9g,5,If you are seeking a cozy Irish bar with excel...,2018-11-14 18:06:16,https://www.yelp.com/biz/chief-sullivans-san-f...,"{'id': '0t-O5--DOfS55A620fO_GA', 'profile_url'...",rest_reviews93497
1,1,Eign91PSSOZ_PI8GLBjEUA,5,So glad to see this place back open. $5 a beer...,2018-10-28 12:25:03,https://www.yelp.com/biz/chief-sullivans-san-f...,"{'id': '9tpv-F4_haLlNO8cJZKZtw', 'profile_url'...",rest_reviews93497


In [41]:
# Remove extra info on file name to get back to business id so it can be matched with other yelp info
df_reviews['new'] = df_reviews['new'].str[12:]

In [42]:
df_reviews.head()

,Unnamed: 0,id,rating,text,time_created,url,user,new
0,0,V1kfikCTAefoyr2K1ljw5Q,5,"I love this place.\n\n\n\nAs of this visit, 5 ...",2018-10-31 18:18:30,https://www.yelp.com/biz/the-beer-hall-san-fra...,"{'id': '4kRxFXdgOulK49RaVVEYow', 'profile_url'...",69249
1,1,B0xxe0oaVd4iakwPbhU5xg,3,Great little place with a nice selection of be...,2018-10-22 08:16:28,https://www.yelp.com/biz/the-beer-hall-san-fra...,"{'id': 'jHreH36KMTpW0eNHA0OexA', 'profile_url'...",69249
2,2,Yd-tAtLR1lIeiO1ugsi4lg,4,"Heaven on earth! Endless taps, diverse options...",2019-03-05 17:53:40,https://www.yelp.com/biz/the-beer-hall-san-fra...,"{'id': 'jJo2YqOoKYTz2sUFBfLChw', 'profile_url'...",69249
0,0,iF2hUCKl1KPT359abomQ9g,5,If you are seeking a cozy Irish bar with excel...,2018-11-14 18:06:16,https://www.yelp.com/biz/chief-sullivans-san-f...,"{'id': '0t-O5--DOfS55A620fO_GA', 'profile_url'...",93497
1,1,Eign91PSSOZ_PI8GLBjEUA,5,So glad to see this place back open. $5 a beer...,2018-10-28 12:25:03,https://www.yelp.com/biz/chief-sullivans-san-f...,"{'id': '9tpv-F4_haLlNO8cJZKZtw', 'profile_url'...",93497
